# Secondary Structure Graphs & Chain Graphs
In this tutorial, we'll be looking at higher-order graphs - that is graphs of Secondary Structure Elements and interacting chains (quaternary structure) - as well as assorted visualisation tools.

Graphein provides utilities for computing these graphs from residue-level graphs. I'd recommend checking out [this tutorial]() for more information.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-r-j/graphein/blob/master/notebooks/higher_order_graphs.ipynb)


First, we'll construct a graph to work with



In [ ]:
# Install Graphein if necessary
# !pip install graphein

# Install DSSP if necessary
# !sudo apt-get install dssp (better for colab) OR !conda install -c salilab dssp

In [ ]:
import logging
import networkx as nx
import matplotlib.pyplot as plt
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("graphein").setLevel(logging.INFO)

import graphein.protein as gp

new_funcs = {
            "edge_construction_functions": [gp.add_peptide_bonds,
                                              gp.add_hydrogen_bond_interactions,
                                              gp.add_disulfide_interactions,
                                              gp.add_ionic_interactions,
                                              gp.add_aromatic_interactions,
                                              gp.add_aromatic_sulphur_interactions,
                                              gp.add_cation_pi_interactions],
             "dssp_config": gp.DSSPConfig()
            }
config = gp.ProteinGraphConfig(**new_funcs)
g = gp.construct_graph(config=config, pdb_code='4hhb')
gp.plotly_protein_structure_graph(g, node_size_multiplier=1)

## Secondary Structure Graphs
We use [DSSP] to compute secondary structures. This is stored under `d["ss"]` in the node attributes. Eg:

In [ ]:
g = gp.secondary_structure(g)
for n, d in g.nodes(data=True):
    print(d["ss"])
    if d["ss"] == "T":
        break

We can compute either a multigraph, where each edge in the selection is preserved (e.g. multiple edges between nodes ``H1`` and ``H2`` are permitted), or a weighted graph where we aggregate multiple edges between a pair of nodes into and preserve the edge information as metadata.

### MultiGraph

In [ ]:
ss_graph = gp.compute_secondary_structure_graph(g, remove_self_loops=True)
print(nx.info(ss_graph))

for u, v, d in g.edges(data=True):
    print(u, v, d)

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)
gp.plot_chord_diagram(ss_graph)

In [ ]:
ss_graph = gp.compute_secondary_structure_graph(g, remove_self_loops=True, return_weighted_graph=True)
print(nx.info(ss_graph))
for u, v, d in ss_graph.edges(data=True):
    print(u, v, d)

## Chain Graph
Chain graphs represent the quaternary structure of a protein complex; nodes represent individual chains and edges are the interactions between them. Again, we can compute either a multigraph or a weighted graphs.

### Multigraph

In [ ]:
g = gp.construct_graph(config=config, pdb_code='2y69')
chain_graph = gp.compute_chain_graph(g, remove_self_loops=False, return_weighted_graph=False)

mat = nx.adjacency_matrix(chain_graph)
a = gp.visualisation.chord_diagram(mat, names=list(chain_graph.nodes), use_gradient=True)

for i, (u, v, d) in enumerate(chain_graph.edges(data=True)):
    print(u, v, d)
    if i == 15:
        break

### Weighted Graph

In [ ]:
chain_graph = gp.compute_chain_graph(g, remove_self_loops=False, return_weighted_graph=True)

mat = nx.adjacency_matrix(chain_graph)
a = gp.visualisation.chord_diagram(mat, names=list(chain_graph.nodes), use_gradient=True)

for u, v, d in chain_graph.edges(data=True):
    print(u,v, d)